Scrape [TV by the numbers](https://tvbythenumbers.zap2it.com/category/daily-ratings/) to get daily ratings for tv shows

In [4]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from RT_Functions import *

In [27]:
error_data = update_rt_data()

138 pages already scraped detected
322 pages needing scraping detected
Successfully scraped with input https://www.rottentomatoes.com/tv/regular_show
Successfully scraped with input https://www.rottentomatoes.com/tv/mr_inbetween
Successfully scraped with input https://www.rottentomatoes.com/tv/elena_of_avalor
Successfully scraped with input https://www.rottentomatoes.com/tv/if_loving_you_is_wrong
Successfully scraped with input https://www.rottentomatoes.com/tv/muppet_babies
Successfully scraped with input https://www.rottentomatoes.com/tv/csi_cyber
Successfully scraped with input https://www.rottentomatoes.com/tv/on_the_case_with_paula_zahn
Failed to fetch data; terminating with error


ReadTimeout: HTTPSConnectionPool(host='www.rottentomatoes.com', port=443): Read timed out. (read timeout=5)

In [17]:
error_data.keys()

dict_keys(['response', 'soup'])

In [ ]:
! conda list '(beautifulsoup4|mysql-connector|pandas|python$)'

In [21]:
soup = error_data['soup']

In [22]:
tv_show_info(soup, url = 'hi')

NameError: name 'url' is not defined

```python
beautifulsoup4            4.8.0                    py37_0    conda-forge
ipython                   7.8.0            py37h5ca1d4c_0    conda-forge
msgpack-python            0.6.2            py37h770b8ee_0    conda-forge
mysql-connector-c         6.1.11            h42c63d9_1003    conda-forge
mysql-connector-python    8.0.18           py37he91358f_0    conda-forge
pandas                    0.25.1           py37h86efe34_0    conda-forge
python                    3.7.3                h359304d_0  
```

In [40]:
from tvbythenumbers import *
import tvbythenumbers as debug #Access to hidden modules for debugging convenience

Definitions:
 - A 'page' contains a number of links (usually 12) to 'articles'
 - An 'article' contains ratings
 
Notes:
 - There are 351 pages, with numbering starting at 1

In [2]:
data = query('''SELECT * FROM tv_by_the_numbers_articles''')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108341 entries, 0 to 108340
Data columns (total 10 columns):
time             108341 non-null object
show             108341 non-null object
demo_rating      107238 non-null object
viewers          108341 non-null object
tags             108341 non-null object
article_title    108341 non-null object
url              108341 non-null object
notes            108341 non-null object
network          71738 non-null object
table_id         108341 non-null int64
dtypes: int64(1), object(9)
memory usage: 8.3+ MB


In [ ]:
data.to_sql('articles_backup_2', db, 'tvshows', index = False, if_exists = 'replace')

False

## Part 1 – Scrape the landing pages to find links to articles

In [ ]:
# Searchs for any pages not currently in the database
#  and puts them in the database. (~idempotent)
# 
# Safe to re-run
# Verbose
# On failure:
#  - returns latest web call data fetched or processed in a dictionary (for debugging purposes)
#  - may raise an error if there is no data to protect
error_data = update_pages_database()

In [ ]:
error_data = update_articles_database(sleep_time = 0.05, on_fail = 'continue')

In [5]:
# Some pages (30) were not fetched
#  On examination: some are due to parsing errors
#                  many are articles not containing data at all
to_do = get_missing_articles()

4176 pages already scraped detected
30 pages needing scraping detected


In [2]:
add_primary_key('tv_by_the_numbers_articles')

In [2]:
def process_broadccast_networks():
    to_alter = query(f'''
        SELECT
            *
        FROM
            tv_by_the_numbers_articles
        WHERE
            network IS NULL
    ''')

    print(f'{len(to_alter)} bad records found')
    broadcast_networks = ['CBS', 'FOX', 'ABC', 'The CW', 'NBC', 'CW', 'NCBS']
    pattern = '(.*)' + '\((' + '|'.join(broadcast_networks) + ')\).*'
    def parse_show(show):
        maybe_match = re.match(pattern, show)
        if not maybe_match:
            out_dict = {'show' : show, 'network' : None}
        else:
            show = maybe_match.group(1)
            net  = maybe_match.group(2)
            if net == 'CW':
                net = 'The CW'
            out_dict = {'show' : show, 'network' : net}
            
        return pd.Series(out_dict)

    fixed = to_alter.show.apply(parse_show)
    
    to_alter['show'] = fixed.show
    to_alter['network'] = fixed.network
    
    to_alter.dropna(subset = ['network'], inplace = True)
    
    ids_to_drop = to_alter.table_id.tolist()
    fixed = to_alter.drop('table_id', axis = 'columns')
    
    
    
    fixed.to_sql('tv_by_the_numbers_articles', db, schema = 'tvshows', if_exists = 'append', index = False)
    
    return ids_to_drop
    try:
        query_text = [
         'DELETE FROM',
         'tv_by_the_numbers_articles',
         'WHERE',
         'table_id',
         'IN',
         str(tuple(ids_to_drop))
        ]
        return '\n'.join(query_text)
    except:
        return ids_to_drop


In [ ]:
a = process_broadccast_networksoadccast_networksbroadccast_networksess_broadcast_networks()

In [ ]:
b = copy.copy(a)

In [17]:
# This is miserably, miserably slow
fail_count = 0
while len(a) > 0:
    try:
        to_delete = a[:100]
        db.execute(f'DELETE FROM tv_by_the_numbers_articles WHERE table_id IN {tuple(to_delete)}')
        for table_id in to_delete:
            a.remove(table_id)
    except:
        print(f'failure on id {a[0]}')
        fail_count = fail_count + 1
        if fail_count > 10:
            print('aborting')
            break
        else:
            continue

In [15]:
len(a)

31340

In [16]:
len(b)

36579

In [2]:
import mysql.connector 


In [5]:
db_name = 'tvshows'
cnx = mysql.connector.connect(
    host = "mon-dragon.cwxehvwsg5bg.us-east-1.rds.amazonaws.com",
    user = "findlay",
    password = "tvshow-*",
    database = db_name,
    use_pure = True
)
cursor = cnx.cursor() 

cursor.execute('''SELECT s.show FROM tv_by_the_numbers_articles s LIMIT 100''')
list_of_tv_shows = cursor.fetchall()


In [6]:
list_of_tv_shows

[('NBA REGULAR SEASON: UTAH/LA LAKERS',),
 ('NBA REGULAR SEASON: DALLAS/NEW ORLEANS',),
 ('GOLD RUSH',),
 ('LIVE PD',),
 ('SPORTSCENTER 1AM',),
 ('GOLD RUSH: SPECIAL',),
 ('BLACK JESUS',),
 ('NBA COURTSIDE',),
 ('WWE FRI NIGHT SMACKDOWN',),
 ('LOVE AFTER LOCKUP',),
 ('REAL TIME WITH BILL MAHER',),
 ('RACHEL MADDOW SHOW',),
 ('PIRATES OF THE CARIBBEAN: DEAD MEN TELL NO TALES',),
 ('LIVE PD AC',),
 ('FIVE, THE',),
 ('NBA PRE-GAME',),
 ('MOM',),
 ('COLLEGE FOOTBALL PRIME: USC/COLORADO',),
 ('DINERS, DRIVE INS & DIVES',),
 ('PARDON THE INTERRUPTION',),
 ('HANNITY SPECIAL',),
 ('TUCKER CARLSON SPECIAL',),
 ('WHAT YOU GET FOR YOUR MONEY',),
 ('FIRST TAKE',),
 ('BEACH HUNTERS',),
 ('NBA BASKETBALL: CLIPPERS/WARRIORS',),
 ('NBA BASKETBALL: BUCKS/ROCKETS',),
 ('NFLN THU NT FOOTBALL: WAS VS. MIN',),
 ('JERSEY SHORE: FAMILY VACATION',),
 ('NFLN THU NT PRE-KICK',),
 ('INSIDE THE NBA',),
 ('FLIP OR FLOP',),
 ('LIVE PD PRESENTS: PD CAM',),
 ('MLS PLAYOFFS',),
 ('LIVE PD PRESENTS: PD CAM',),
 ('FLIP 

In [7]:
string_list_of_tv_shows = [''.join(i) for i in list_of_tv_shows] 


In [ ]:
string_list_of_tv_shows

In [1]:
import requests
from bs4 import BeautifulSoup as BS
import time
import mysql.connector


In [4]:

db_name = 'tvshows'
cnx = mysql.connector.connect(
    host = "mon-dragon.cwxehvwsg5bg.us-east-1.rds.amazonaws.com",
    user = "findlay",
    password = "tvshow-*",
    database = db_name,
    use_pure = True
)
cursor = cnx.cursor()

cursor.execute('''SELECT s.show FROM tv_by_the_numbers_articles s LIMIT 100''')
list_of_tv_shows = cursor.fetchall()
string_list_of_tv_shows = [''.join(i) for i in list_of_tv_shows]

In [37]:
import tvbythenumbers as utilities

In [64]:
def find_on_rotten_tomatoes(list_of_shows):
    list_of_rotten_tomatoes_urls = []
    def fix_name(s):
        allowed_characters = string.ascii_lowercase + ' _'
        s = ''.join(filter(lambda c : c in allowed_characters, s.lower()))
        s = s.replace(' ','_')
        return s

    for i, show in enumerate(list_of_shows):
        if i % 10 == 0:
            print(f'Attempting show number {i}')
        try:
            rt_name = fix_name(show)
            r = requests.get(f'https://www.rottentomatoes.com/tv/{rt_name}')
            if r.status_code == 200:
                url = f'https://www.rottentomatoes.com/tv/{rt_name}'
                out_dict = {'by_the_numbers_name' : show,
                            'rt_name'             : rt_name,
                            'url'                 : url}
                list_of_rotten_tomatoes_urls.append(out_dict)
        except Exception as e:
            print('------SOMETHING WENT HORRIBLY WRONG-----')
            print(f'show was {show}')
            print(e)
            time.sleep(1)
            continue

        time.sleep(0.05)

    return list_of_rotten_tomatoes_urls

In [69]:
list_of_urls = find_on_rotten_tomatoes(list_of_tv_shows[400:])

Attempting show number 0
Attempting show number 10
Attempting show number 20
Attempting show number 30
Attempting show number 40
Attempting show number 50
Attempting show number 60
Attempting show number 70
Attempting show number 80
Attempting show number 90
Attempting show number 100
Attempting show number 110
Attempting show number 120
Attempting show number 130
Attempting show number 140
Attempting show number 150
Attempting show number 160
Attempting show number 170
Attempting show number 180
Attempting show number 190
Attempting show number 200
Attempting show number 210
Attempting show number 220
Attempting show number 230
Attempting show number 240
Attempting show number 250
Attempting show number 260
Attempting show number 270
Attempting show number 280
Attempting show number 290
Attempting show number 300
Attempting show number 310
Attempting show number 320
Attempting show number 330
Attempting show number 340
Attempting show number 350
Attempting show number 360
Attempting s

Attempting show number 2970
Attempting show number 2980
Attempting show number 2990
Attempting show number 3000
Attempting show number 3010
Attempting show number 3020
Attempting show number 3030
Attempting show number 3040
Attempting show number 3050
Attempting show number 3060
Attempting show number 3070
Attempting show number 3080
Attempting show number 3090
Attempting show number 3100
Attempting show number 3110
Attempting show number 3120
Attempting show number 3130
Attempting show number 3140
Attempting show number 3150
Attempting show number 3160
Attempting show number 3170
Attempting show number 3180
Attempting show number 3190
Attempting show number 3200
Attempting show number 3210
Attempting show number 3220
Attempting show number 3230
Attempting show number 3240
Attempting show number 3250
Attempting show number 3260
Attempting show number 3270
Attempting show number 3280
Attempting show number 3290
Attempting show number 3300
Attempting show number 3310
Attempting show numb

Attempting show number 5900
Attempting show number 5910
Attempting show number 5920
Attempting show number 5930
Attempting show number 5940
Attempting show number 5950
Attempting show number 5960
Attempting show number 5970
Attempting show number 5980
Attempting show number 5990
Attempting show number 6000
Attempting show number 6010
Attempting show number 6020
Attempting show number 6030
Attempting show number 6040
Attempting show number 6050
Attempting show number 6060
Attempting show number 6070
Attempting show number 6080
Attempting show number 6090
Attempting show number 6100
Attempting show number 6110
Attempting show number 6120
Attempting show number 6130
Attempting show number 6140
Attempting show number 6150
Attempting show number 6160
Attempting show number 6170
Attempting show number 6180
Attempting show number 6190
Attempting show number 6200
Attempting show number 6210
Attempting show number 6220
Attempting show number 6230
Attempting show number 6240
Attempting show numb

In [70]:
df = pd.DataFrame.from_records(list_of_urls)

In [71]:
df.to_sql('rt_urls', db,
          index = False, if_exists = 'append')


In [72]:
df

,by_the_numbers_name,rt_name,url
0,SNOWFALL,snowfall,https://www.rottentomatoes.com/tv/snowfall
1,QUEEN SUGAR,queen_sugar,https://www.rottentomatoes.com/tv/queen_sugar
2,GROWING UP CHRISLEY,growing_up_chrisley,https://www.rottentomatoes.com/tv/growing_up_c...
3,DEADLIEST CATCH,deadliest_catch,https://www.rottentomatoes.com/tv/deadliest_catch
4,GHOSTED,ghosted,https://www.rottentomatoes.com/tv/ghosted
...,...,...,...
536,What Would You Do?,what_would_you_do_,https://www.rottentomatoes.com/tv/what_would_y...
537,Whose Line is it Anyway,whose_line_is_it_anyway_,https://www.rottentomatoes.com/tv/whose_line_i...
538,"Alex, Inc.",alex_inc_,https://www.rottentomatoes.com/tv/alex_inc_
539,American Gothic,american_gothic,https://www.rottentomatoes.com/tv/american_gothic


In [68]:
len(list_of_urls)

26